# Faster RCNN

## Components


### Region Proposal Network (RPN)
- input: image of any size
- output: set of rectangular object proposals, each w/ an "objectness" score
- black box: 
    - n x n spatial window slides across feature map output
        - ie. n x n convolutional layer
        - w/ n=3, ZF/VGG receptive field size is 171/228 pixels
    - window maps to lower-dimensional feature (256-d/512-d w/ ReLU)
    - feed feature into 2 sibling FCNs
        - Box-regression layer (reg)
        - Box-classification layer (cls)
        - ie. 1 x 1 convolutional layers
    - at each sliding-window location, predict multiple proposals given number of anchors k
        - therefore, reg layer has 4k outputs encoding coordinates
            - parameterize w.r.t k reference boxes
        - therefore, cls layer has 2k outputs estimating probability of object or not object
        - each anchor has specified scale and aspect ratio
            - use 3 scales, 3 aspect ratios for k = 9 anchors
        - translation-invariant anchors:
            - regress bounding boxes w/ ref to anchor boxes
    - loss function:
        - for training, assign binary class label (obj or not) to each anchor
            - Positive:
                - anchor/anchors w/ highest IoU overlap w/ ground-truth box (rare case)
                - OR
                - anchor that has IoU overlap higher than 0.7 with any ground-truth box (most common case)
            - single ground-truth box may assign positive labels to multiple anchors
            - Negative:
                - non-positive anchor if its IoU ratio is lower than 0.3 for all ground-truth boxes
            - Neither:
                - then anchors don't contribute to training
        - Loss Function defined in paper
            - Lcls is log loss over two classes
            - Lreg is robust loss function (ie. smooth L1)
            - regression loss activated only for positive anchors
            - normalization by scale factor (optional)
        - bbox parameterization:
            - define w/ log and distance from corners of bbox and size of box
            - essentially, regression from anchor box to nearby ground-truth box
            - translation-invariant
    - Training RPNs
        - stochastic gradient descent
        - each mini-batch arises from single image containing many positive and negative anchors
        - to prevent bias towards the more common negative anchors, randomly sample 256 anchors in image w/ up to 1:1 ratio
            - if fewer than 128 positive samples, pad mini-batch w/ negative ones
        - randomly initialize all new layers by drawing weights form zero-mean Gauss distrib. w/ std of 0.01
            - all other layers (ie. shared conv layers) are initialized by pretraining model for ImageNet classification (ie. fine-tune ZF, VGG, etc.)
        - lr = 0.001 for 60k mini-batches
        - lr = 0.0001 for next 20k mini-batches
        - momentum = 0.9
        - weight decay = 0.0005

## Training: 4-Step Alternating Training
1. fine-tune pre-trained ImageNet model for RPN
2. train separate detection network by Fast R-CNN using generated proposals from step-1 (also a pre-trained ImageNet model)
3. use detector network to initialize RPN training, but fix the shared conv layers and only fine-tune the layers unique to RPN (now two networks share conv layers)
4. keeping shared layers fixed, fine-tune unique layers of Fast R-CNN


## Training: Approximate joint training

## Training: Non-approximate joint training

### Fast R-CNN object detection network